### 1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
import numpy as np

In [85]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, y_pred - 1e-100, y_pred) # Смещение в меньшую сторону для единиц
    y_pred = np.where(y_pred == 0, y_pred + 1e-100, y_pred) # Смещение в большую сторону для нулей
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [86]:
# Проверка 1
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [87]:
# Проверка 2
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [94]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [95]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [96]:
W = eval_model(X_st, y, iterations=500, eta=1e-4)

0 [ 0.49667621 -0.13840939  0.6476858   1.52297324] 1.1785958344356262
50 [ 0.494784   -0.14564801  0.6475462   1.52014828] 1.1657985749255426
100 [ 0.49290109 -0.15285535  0.64740132  1.51733474] 1.1531112685708473
150 [ 0.49102761 -0.16003088  0.64725118  1.51453281] 1.1405352753305018
200 [ 0.48916364 -0.16717404  0.64709581  1.51174267] 1.1280719326917483
250 [ 0.48730929 -0.17428428  0.64693524  1.50896452] 1.1157225565960736
300 [ 0.48546465 -0.18136107  0.64676951  1.50619853] 1.103488442622439
350 [ 0.48362982 -0.18840385  0.64659868  1.5034449 ] 1.0913708674192037
400 [ 0.48180488 -0.19541206  0.64642281  1.50070383] 1.0793710903721336
450 [ 0.47998993 -0.20238516  0.64624195  1.49797551] 1.0674903554915993


In [110]:
W = eval_model(X_st, y, iterations=5000000, eta=1e-2)

0 [ 0.49292028 -0.15277306  0.64741473  1.51736839] 1.1785958344356262
500000 [-24.52628686  -2.59421335  -6.46566094  20.31253756] 0.14388121210374644
1000000 [-33.59339885  -3.41350075  -9.03354451  27.65513226] 0.11461929785948144
1500000 [-39.81594454  -3.96618617 -10.78934067  32.65730829] 0.1011022070392742
2000000 [-44.75777387  -4.39218777 -12.19158314  36.59709587] 0.09265889648993494
2500000 [-48.9801316   -4.74310416 -13.40022964  39.93384042] 0.08654035385896659
3000000 [-52.74490665  -5.04358331 -14.48849362  42.88263961] 0.0817074505707888


<ipython-input-85-b4f15789c917>:4: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
<ipython-input-85-b4f15789c917>:4: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


KeyboardInterrupt: 

Ход решения:
- iters = 5000 - ошибка продолжает уменьшаться
- iters = 50000 - ошибка продолжает уменьшаться
- iters = 500000 - ошибка продолжает уменьшаться и веса значительно изменяются
- iters = 500000, eta=1e-3 - ошибка продолжает уменьшаться и веса значительно изменяются
- iters = 500000, eta=1e-2 - ошибка продолжает уменьшаться и веса значительно изменяются; подозрение на переобучение, т.к. веса сильно расходятся
- iters = 500000, eta=1e-1: err = 0.08170729719956263 (iter=300000) ошибка продолжает уменьшаться и веса значительно изменяются. В np.log после 300000 стали попадать невалидные значения. Наименьшее err где-то между 300000 и 350000. Пробую в 10 раз увеличить количество итераций и в 10 раз уменьшить скорость.
- iters = 5000000, eta=1e-2: err = 0.0817074505707888 (iter=3000000). Ошибка увеличилась. Модифицирую calc_logloss для большего смещения. eval_model для задания стартовых значений весов и вывода итоговой ошибки.

In [118]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, y_pred - 1e-10, y_pred) # Смещение в меньшую сторону для единиц
    y_pred = np.where(y_pred == 0, y_pred + 1e-10, y_pred) # Смещение в большую сторону для нулей
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

def eval_model_W(W, X, y, iterations, eta):
    np.random.seed(42)
    n = X.shape[0]
    err = 0
    
    for i in range(iterations):
        try:
            z = np.dot(X, W)
            y_pred = sigmoid(z)
            err = calc_logloss(y, y_pred)
        
            dQ = 1/n * X.T @ (y_pred - y)
            W -= eta * dQ
            
            if i % (iterations / 10) == 0:
                print(i, W, err)
        except:
            print(f"min error: {err}")
    return W, err

In [119]:
W = [-52.74490665, -5.04358331, -14.48849362, 42.88263961]

In [120]:
eval_model_W(W, X_st, y, iterations=2000000, eta=1e-2)

0 [-52.74491383  -5.04358387 -14.48849571  42.88264521] 0.08170744181779505
200000 [-54.15674273  -5.15297007 -14.89943525  43.98170397] 0.08001626231922183
400000 [-55.52465586  -5.25718655 -15.29909976  45.04301554] 0.07843205212263596
600000 [-56.85361728  -5.35676538 -15.68879934  46.07077723] 0.07693984207098545
800000 [-58.14774552  -5.45216637 -16.06960417  47.06850529] 0.07552758761267964
1000000 [-59.41048257  -5.54379135 -16.44239339  48.03917023] 0.07418548856315706
1200000 [-60.6447235   -5.63199464 -16.80789288  48.98529994] 0.07290549136077311
1400000 [-61.85291748  -5.71709073 -17.16670484  49.90905924] 0.07168091842869685
1600000 [-63.03714764  -5.79936015 -17.51933134  50.81231232] 0.07050618783326949
1800000 [-64.19919479  -5.87905381 -17.86619315  51.69667222] 0.06937659823930724


(array([-65.34058333,  -5.95639614, -18.20764331,  52.56353636]),
 0.06828816721821737)

Вывод: err продолжает падать. Модель явно подгоняется под изначальные данные и переобучается. Для такого малого набора данных дальнейшее обучение модели бесполезно.

Минимальная ошибка на обучающей выборке: 0.06828816721821737 при iters = 5000000 и eta=1e-2.

### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

$$p_{+} = \sigma(\left \langle w,x_{i} \right \rangle) = \frac{1}{1 + exp(-\left \langle w,x_{i} \right \rangle)},$$

In [152]:
W1 = [-64.19919479, -5.87905381, -17.86619315, 51.69667222]
W1, X

([-64.19919479, -5.87905381, -17.86619315, 51.69667222],
 array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
        [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
        [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
        [1.00e+00, 5.00e+00, 6.00e+02, 1.00e+00],
        [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
        [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
        [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
        [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
        [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
        [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]]))

In [195]:
def calc_pred_proba(W, X):
    return 1 / (1 + np.exp(- np.dot(W, X.T)))

In [197]:
y_pred_proba = calc_pred_proba(W1, X)
y_pred_proba

<ipython-input-195-c1c6d1a728aa>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(- np.dot(W, X.T)))


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [198]:
W2 = [-24.52628686, -2.59421335, -6.46566094, 20.31253756]
calc_pred_proba(W2, X)

<ipython-input-195-c1c6d1a728aa>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(- np.dot(W, X.T)))


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [199]:
W3 = [0.47998993, -0.20238516, 0.64624195, 1.49797551]
calc_pred_proba(W3, X)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [200]:
W4 = [0.49667621, -0.13840939, 0.6476858, 1.52297324]
calc_pred_proba(W4, X)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [201]:
eval_model(X_st, y, iterations=1, eta=1e-2)

0 [ 0.49292028 -0.15277306  0.64741473  1.51736839] 1.1785958344356262


array([ 0.49292028, -0.15277306,  0.64741473,  1.51736839])

In [202]:
W5 = [0.49292028, -0.15277306, 0.64741473, 1.51736839]
calc_pred_proba(W5, X)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Вывод: из полученных результатов видно, что модель строится сразу так, что предсказанная вероятность класса 1 всегда равна 1.
Странно, что не получилось добиться меньшей вероятности.

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [166]:
def calc_pred(W, X):
    m = X.shape[0]
    y_predicted = np.zeros(m)
    y_pred_proba = 1 / (1 + np.exp(- np.dot(W, X.T)))
    for i in range(m):
        if y_pred_proba[i] > 0.5:
            y_predicted[i] = 1
    return y_predicted

calc_pred(W5, X)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Подбор весов вручную для проверки работоспособности методов.

In [203]:
W6 = [0, 0.1, 0, -0.1]
calc_pred(W6, X)

array([0., 0., 0., 1., 1., 0., 1., 1., 0., 0.])

In [204]:
calc_pred_proba(W6, X)

array([0.5       , 0.5       , 0.5       , 0.59868766, 0.52497919,
       0.47502081, 0.549834  , 0.66818777, 0.5       , 0.5       ])

Вывод: удалось добиться вероятностей между 0 и 1. Вероятно, методы правильные.

# 6. Могла ли модель переобучиться? Почему?

Модель явно переобучена, т.к. из задачи 2 все вероятности сразу равны 1.